In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('videos.csv')
df

,pK,title,channel_ID,category,duration,rating
0,1,2 hours soft rain sounds,1,background,2 hours,6.8
1,2,New Tetris?!?!,2,gaming,30 min,8.0
2,3,My apology,2,discussion,11 min,3.4
3,4,Dog TikTok Compilation,3,animals,7 min,8.7
4,5,Not A Rickroll,4,music,3 min,2.6
5,6,Learn MongoDB,5,educational,1 hour,5.2
6,7,Learn Python in 30 seconds!,5,educational,30 sec,8.9
7,8,Let's Play Tears of the Kingdom,2,gaming,2 hours,7.5
8,9,How to Travel for Cheap,6,discussion,15 min,8.5
9,10,Big Announcement...,4,discussion,17 min,9.0


In [3]:
df['duration'] = pd.to_timedelta(df['duration'])
df['duration'] = df['duration'] / pd.Timedelta(minutes=1)


# for i in df['duration']:
#     print(i, pd.to_timedelta(i))
df

,pK,title,channel_ID,category,duration,rating
0,1,2 hours soft rain sounds,1,background,120.0,6.8
1,2,New Tetris?!?!,2,gaming,30.0,8.0
2,3,My apology,2,discussion,11.0,3.4
3,4,Dog TikTok Compilation,3,animals,7.0,8.7
4,5,Not A Rickroll,4,music,3.0,2.6
5,6,Learn MongoDB,5,educational,60.0,5.2
6,7,Learn Python in 30 seconds!,5,educational,0.5,8.9
7,8,Let's Play Tears of the Kingdom,2,gaming,120.0,7.5
8,9,How to Travel for Cheap,6,discussion,15.0,8.5
9,10,Big Announcement...,4,discussion,17.0,9.0


In [64]:
from py2neo import Graph, Node, Relationship, NodeMatcher

matcher = NodeMatcher(graph)
# matcher.match("Video").where("_.pK = '3'")

In [65]:
channels = pd.read_csv('channels.csv')
views = pd.read_csv('views.csv')
subs = pd.read_csv('subs.csv')

In [66]:
graph = Graph("bolt://localhost:7687", auth=("neo4j", "password"))
graph.run("Match (n) detach delete n")

(No data)

In [67]:
for i in range(len(channels)):
    row = channels.iloc[i]
    # print(row)
    node = Node("Channel", pK = int(row['pK']), name = row['name'], subscribers = int(row['subscribers']))
    graph.create(node)

In [68]:
for i in range(len(df)):
    row = df.iloc[i]
    # print(row)
    vid = Node("Video", pK = int(row['pK']), title = row['title'], duration = float(row['duration']))
    graph.create(node)
    channel = matcher.match("Channel").where(f"_.pK = {row['channel_ID']}").first()
    # print(channel)
    graph.create(Relationship(channel, "HAS_VIDEO", vid))

In [62]:
for i in range(len(views)):
    row = views.iloc[i]
    # print(row)
    view = Node("View", user_ID = int(row['user_ID']), duration_watched=float(row['duration_watched']))
    graph.create(view)
    video = matcher.match("Video").where(f"_.pK = {row['video_ID']}").first()
    graph.create(Relationship(view, "VIEWED", video))

In [63]:
for i in range(len(subs)):
    row = subs.iloc[i]
    # print(row)
    sub = Node("Subscription", user_ID = int(row['user_ID']))
    graph.create(sub)
    channel = matcher.match("Channel").where(f"_.pK = {row['channel_ID']}").first()
    # print(channel)
    graph.create(Relationship(sub, f"{row['status']}_{row['subscription_date']}", channel))